<a href="https://colab.research.google.com/github/samir141673/paralelas-2020-1/blob/main/practica_2_Taxonomia_de_Flynn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

El siguiente código va a permitir que todo código ejecutado en el colab pueda ser medido

In [ ]:
!pip install ipython-autotime

%load_ext autotime

time: 1.38 ms


In [ ]:
print(sum(range(10)))

45
time: 968 µs


Pregunta #1: Que porción de 1 segundo es el valor impreso?


 es milisegundo


---



A seguir, tenemos una librería de Python llamado **numba** que realiza paralelización automatica. Asi, se puede verificar que al usar prange() se tiene mejor tiempo de ejecución que al usar range()

In [ ]:
from numba import njit, prange
import numpy as np

A = np.arange(5, 10)
@njit(parallel=True)
def prange_test(A):
    s = 0
    # Without "parallel=True" in the jit-decorator
    # the prange statement is equivalent to range
    for i in prange(A.shape[0]):
        s += A[i]
    return s

print(prange_test(A))

35
time: 440 ms


In [ ]:
from numba import njit, prange

A = np.arange(5, 10)
#@njit(parallel=True)
def prange_test(A):
    s = 0
    # Without "parallel=True" in the jit-decorator
    # the prange statement is equivalent to range
    for i in range(A.shape[0]):
        s += A[i]
    return s

print(prange_test(A))



35
time: 4.95 ms


Pregunta #2: identifique otros valores en A, de manera que, serializando, tengamos mejor resultado que paralelizando


Cuando ponemos datos de entrada pequenio vemos q el sereal es mas rapido q el paralelo



---



La Taxonomia de Flynn define 4 tipos de arquitecturas para computación paralela: SISD, SIMD, MISD, y MIMD. 


---


Pregunta #3 : El ultimo código ejecutado es de tipo?



---



Pregunta #4: el siguiente código paralelo es de tipo? Comentar el código para justificar su respuesta

simple instruccion multiples datos SIMD

In [ ]:
import threading
import time

def print_time(name, n):
    count = 0   
    print("Para el Hilo: %s, en el momento: %s, su valor de count es: %s" % ( name, time.ctime(), count))
    while count < 5:
        time.sleep(n)
        count+=1
        print("%s: %s. count %s" % ( name, time.ctime(), count))

        
t1 = threading.Thread(target=print_time, args=("Thread-1", 0, ) )
t2 = threading.Thread(target=print_time, args=("Thread-2", 0, ) )

t1.start()
t2.start()

Para el Hilo: Thread-1, en el momento: Wed Dec  9 15:21:16 2020, su valor de count es: 0Para el Hilo: Thread-2, en el momento: Wed Dec  9 15:21:16 2020, su valor de count es: 0
Thread-1: Wed Dec  9 15:21:16 2020. count 1
Thread-1: Wed Dec  9 15:21:16 2020. count 2

Thread-2: Wed Dec  9 15:21:16 2020. count 1time: 17.7 ms

Thread-1: Wed Dec  9 15:21:16 2020. count 3
Thread-1: Wed Dec  9 15:21:16 2020. count 4
Thread-2: Wed Dec  9 15:21:16 2020. count 2
Thread-2: Wed Dec  9 15:21:16 2020. count 3
Thread-2: Wed Dec  9 15:21:16 2020. count 4
Thread-2: Wed Dec  9 15:21:16 2020. count 5
Thread-1: Wed Dec  9 15:21:16 2020. count 5




---



Una computadora paralela tipo MIMD es utilizado más en la computación distribuida, ejm. Clusters. El siguiente código en python desktop muestra tal funcionamiento

In [ ]:
#greeting-server.py
import Pyro4

@Pyro4.expose
class GreetingMaker(object):
    def get_fortune(self, name):
        return "Hello, {0}. Here is your fortune message:\n" \
               "Behold the warranty -- the bold print giveth and the fine print taketh away.".format(name)

daemon = Pyro4.Daemon()                # make a Pyro daemon
uri = daemon.register(GreetingMaker)   # register the greeting maker as a Pyro object

print("Ready. Object uri =", uri)      # print the uri so we can use it in the client later
daemon.requestLoop()                   # start the event loop of the server to wait for calls

In [ ]:
#greeting-client.py
import Pyro4

uri = input("What is the Pyro uri of the greeting object? ").strip()
name = input("What is your name? ").strip()

greeting_maker = Pyro4.Proxy(uri)         # get a Pyro proxy to the greeting object
print(greeting_maker.get_fortune(name))   # call method normally

Pregunta #5: Explique que hace este código de tipo MIMD


**MULTIPLES INSTRUCCIONES**

En en campo del servirdor se pueden registrar varios "URIS" a la vez  dando un registro de login  
**MULTIPLES DATOS**


 para despues desde el lado del cliente se ingrese como datos de  esos logins q ha sido registrados en la servidor, y el nombre del usuario. votando un printi de "Hello, samir. Here is your fortune message:
               "Behold the warranty -- the bold print giveth and the fine print taketh away.""



---



Ejercicio Propuesto: Crear un ejemplo que muestre una computación paralela de tipo MISD

**MISD:** Implica q muchos procesadores aplican diferentes instrucciones al mismo dato , esa posibilidad hipotetica se considera generalente impracticable 

**Características del modelo MISD:**


*Cada unidad ejecuta una instrucción distinta

*Cada unidad procesa el mismo dato

*Aplicación muy limitada en la vida real

Según (JOHN WILEY & SONS, 2004)
dice: “D
ebido a su carácter abstracto puro,ninguna máquina prácticamente viable ha ejemplificado el modelo MISD introducidopor la clasificación (al menos hasta ahora), se ha considerado máquinas pipeline (yquizás computadoras sistólica -Array) como ejemplos para MISD
”

**APLICACIONES**

*Aplicacion de redes neuronales

*Ordenadores de control del transbordador espacial de vuelo

https://www.academia.edu/18404647/Taxonom%C3%ADas_de_Flynn
https://prezi.com/nwy6cbugo8gg/taxonomia-de-flynn/?frame=75b9549413c4c0d04b446bdf1692aadf5cd69577
https://www.ecured.cu/Taxonom%C3%ADa_de_Flynn

**Referencias**

https://wiki.python.org/moin/ParallelProcessing

https://numba.readthedocs.io/en/stable/user/parallel.html

https://ao.gl/how-to-measure-execution-time-in-google-colab/

http://noisymime.org/blogimages/SIMD.pdf